##Filter data

Filter the data in the Iberian Peninsula. Cleaning the data.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from mpl_toolkits.basemap import Basemap
%matplotlib inline

In [2]:
#limits latitud i longitud
# Península
lon_w=-9.536133
lon_e=3.076172
lat_s=35.995785
lat_n=43.961191

In [3]:
iterator_station=pd.read_csv("Dades/DadesInicialsRAW/site_detail.txt",sep="\t",comment="%",chunksize=100000,names=["Station ID", "Station Name", "Latitude", "Longitude", "Elevation (m)", "Lat. Uncertainty", "Long. Uncertainty", "Elev. Uncertainty (m)", "Country", "State / Province Code", "County", "Time Zone", "WMO ID", "Coop ID", "WBAN ID", "ICAO ID", "# of Relocations", "# Suggested Relocations", "# of Sources", "Hash"])
df_station=pd.concat([chunk[(chunk['Latitude'] > lat_s )& (chunk["Latitude"]<lat_n) &(chunk["Longitude"]>lon_w)&(chunk["Longitude"]<lon_e)] for chunk in iterator_station],ignore_index=True)
df_station.head()
stationid=df_station["Station ID"].tolist()

In [4]:
df_station.head()

,Station ID,Station Name,Latitude,Longitude,Elevation (m),Lat. Uncertainty,Long. Uncertainty,Elev. Uncertainty (m),Country,State / Province Code,County,Time Zone,WMO ID,Coop ID,WBAN ID,ICAO ID,# of Relocations,# Suggested Relocations,# of Sources,Hash
0,275,PALMA MALLORCA/SON BONET,39.60000,2.7000,45.00,0.05000,0.05000,0.50,Spain,,,-99,-9999,-9999,-9999,,0,0,1,b11fe408493a0bc939163d666369acf5
1,285,BARCELONA AP EL PRATc,41.29167,2.0750,1.60,0.01667,0.01667,10.00,Spain,,,-99,8181,-9999,-9999,,0,1,1,d1bf8c74789365dc557db040c5486056
2,2752,ORLEANSVILLE,36.17000,1.5000,112.00,0.00500,0.00500,0.05,Algeria,,,-99,-9999,-9999,-9999,,0,0,2,eaa9e882d6a4c6c30e069393b1e7ed88
3,2763,BLIDA ALGERIA,36.49100,2.8165,170.05,0.00850,0.00100,0.05,Algeria,,,-99,-9999,-9999,-9999,,0,0,1,6c5548ac2e0d4220198ac59aa98bb3d3
4,2776,CAP CAXINE,36.81700,2.9500,33.00,0.00050,0.00050,0.05,Algeria,,,-99,-9999,-9999,-9999,,0,0,1,166a6428e2db55cdf10f2cc0928966e3


In [5]:
print df_station.shape

(201, 20)


In [6]:
df_station["Country"].unique()

array(['Spain                                   ',
       'Algeria                                 ',
       'France                                  ',
       'Gibraltar [United Kingdom]              ',
       'Portugal                                ',
       'Turkey                                  '], dtype=object)

In [7]:
#df_station[df_station["Country"]=='Spain                                   ']
spain_stations=df_station[df_station["Country"]=='Spain                                   ']
gibraltar_stations=df_station[df_station["Country"]=='Gibraltar [United Kingdom]              ']
portugal_stations=df_station[df_station["Country"]=='Portugal                                ']

print spain_stations.shape
print gibraltar_stations.shape
print portugal_stations.shape

(134, 20)
(4, 20)
(30, 20)


In [8]:
spain_stations.shape[0]

134

In [9]:
#Passar a stattions les estacions desitjades.

stations=pd.concat([spain_stations, gibraltar_stations, portugal_stations])
new_stationid=stations["Station ID"].tolist()

In [10]:
iterator_data=pd.read_csv("Dades/DadesInicialsRAW/data.txt",sep="\t",comment="%",chunksize=100000,names=["Station ID", "Series Number", "Date", "Temperature (C)", "Uncertainty (C)"," Observations"," Time of Observation"])
df_data=pd.concat([chunk[(chunk["Station ID"].isin(stationid))] for chunk in iterator_data],ignore_index=True)

In [11]:
df_data.head()

,Station ID,Series Number,Date,Temperature (C),Uncertainty (C),Observations,Time of Observation
0,275,1,1951.042,10.2,0.05,-99,-99
1,275,1,1951.125,10.2,0.05,-99,-99
2,275,1,1951.208,11.7,0.05,-99,-99
3,275,1,1951.292,13.4,0.05,-99,-99
4,275,1,1951.375,15.3,0.05,-99,-99


In [12]:
df_data["Station ID"].unique().shape

(201,)

In [13]:
iterator_data=pd.read_csv("Dades/DadesInicialsRAW/data.txt",sep="\t",comment="%",chunksize=100000,names=["Station ID", "Series Number", "Date", "Temperature (C)", "Uncertainty (C)"," Observations"," Time of Observation"])
data=pd.concat([chunk[(chunk["Station ID"].isin(new_stationid))] for chunk in iterator_data],ignore_index=True)

In [14]:
if (data["Station ID"].unique().shape[0]==stations.shape[0]):
    print "OK: Les dimensions coincideixen"
else:
    print "ERROR: Les dimensions no coincideixen"

OK: Les dimensions coincideixen


In [15]:
dates=pd.DataFrame(columns=["Station ID", "minDate", "maxDate", "Period"])

for i in data["Station ID"].unique():
    T=data[data["Station ID"]==i]
    fila=pd.DataFrame([[i, T["Date"].min(), T["Date"].max(), T["Date"].max()-T["Date"].min()]], 
                      columns=["Station ID", "minDate", "maxDate", "Period"])
    dates=dates.append(fila, ignore_index=True)
dates.head()

,Station ID,minDate,maxDate,Period
0,275,1951.042,1970.958,19.916
1,285,2000.708,2013.708,13.000
2,14653,1938.042,1947.958,9.916
3,14654,1852.458,1934.958,82.500
4,14655,1887.042,1937.958,50.916


In [16]:
# Filtrem estacions que no tenen dades posteriors a 1900.

data=data[data["Station ID"].isin(dates[dates["maxDate"]>1900]["Station ID"].values)]
print len(dates[dates["maxDate"]>1900])

#Filtrem estacions que no tenen mes de 3 anys de dades

data=data[data["Station ID"].isin(dates[dates["Period"]>3]["Station ID"].values)]
print len(dates[dates["Period"]>3])

# Treure totes les dades anteriors a 1900

data=data[data["Date"]>1900]
print len(data[data["Date"]>1900])
print len(data)


167
162
88370
88370


In [17]:
data.index=list(range(len(data.index)))
data.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9, 
            ...
            88360, 88361, 88362, 88363, 88364, 88365, 88366, 88367, 88368,
            88369],
           dtype='int64', length=88370)

In [18]:
# Add year and month in data

year=pd.DataFrame(data=np.floor(data["Date"]).values, columns=["Year"])
month=pd.DataFrame(np.floor(((data["Date"]-np.floor(data["Date"])-1/24.0)*12+1).round(1)).values, columns=["Month"])
data=pd.concat([data, year, month], axis=1)

In [19]:
stations.to_csv("Dades/OUTPUT 01 Filtered Stations.csv", index=False)
data.to_csv("Dades/OUTPUT 01 Filtered Dades Peninsula.csv", index=False)

In [20]:
data=pd.read_csv("Dades/OUTPUT 01 Filtered Dades Peninsula.csv")
data

,Station ID,Series Number,Date,Temperature (C),Uncertainty (C),Observations,Time of Observation,Year,Month
0,275,1,1951.042,10.200,0.0500,-99,-99,1951,1
1,275,1,1951.125,10.200,0.0500,-99,-99,1951,2
2,275,1,1951.208,11.700,0.0500,-99,-99,1951,3
3,275,1,1951.292,13.400,0.0500,-99,-99,1951,4
4,275,1,1951.375,15.300,0.0500,-99,-99,1951,5
5,275,1,1951.458,21.500,0.0500,-99,-99,1951,6
6,275,1,1951.542,23.800,0.0500,-99,-99,1951,7
7,275,1,1951.625,24.100,0.0500,-99,-99,1951,8
8,275,1,1951.708,22.400,0.0500,-99,-99,1951,9
9,275,1,1951.792,16.700,0.0500,-99,-99,1951,10
